In [1]:
import numpy as np
import pandas as pd
import zipfile
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [1]:
#pip install implicit

     ------------------------------------ 752.1/752.1 kB 312.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
zip_path = '../00_Data_Bases/Cluster5_1_items.zip' 
csv_filename = 'Cluster5_1_items.csv'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open(csv_filename) as f:
        df = pd.read_csv(f)

probar con mas variables

In [3]:
df.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name,day,order_time_list,max_order,order_number_group,Clusters_5_1_PCA
0,2766469,58222,3,6,11,6.0,53,1,1,16,dairy eggs,cream,Sunday,Morning,3,1-20 order,0
1,2766469,58222,3,6,11,6.0,86,2,1,16,dairy eggs,eggs,Sunday,Morning,3,1-20 order,0
2,2766469,58222,3,6,11,6.0,115,3,1,7,beverages,water seltzer sparkling water,Sunday,Morning,3,1-20 order,0
3,2766469,58222,3,6,11,6.0,120,4,0,16,dairy eggs,yogurt,Sunday,Morning,3,1-20 order,0
4,2766469,58222,3,6,11,6.0,17,5,0,13,pantry,baking ingredients,Sunday,Morning,3,1-20 order,0


### Matriz de Interaccion


In [ ]:
# Optimizar la métrica de interacción (score)
df["score"] = (
    df["reordered"] * 2  # Dar más peso a recompras
    + 1  # Sumar 1 a toda interacción
    + (df["add_to_cart_order"] <= 5).astype(int)  # Priorizamos productos agregados primero al carrito
    + (df["days_since_prior_order"] <= 7).astype(int)  # Priorizamos compras recientes
)


In [4]:
# Filtrar usuarios y productos con interacciones válidas
valid_users = df["user_id"].unique()
valid_products = df["product_id"].unique()

# Crear mapeos asegurando alineación con la matriz
user_map = {u: i for i, u in enumerate(valid_users)}
item_map = {i: j for j, i in enumerate(valid_products)}

# Aplicar mapeo a los datos
df["user_index"] = df["user_id"].map(user_map)
df["item_index"] = df["product_id"].map(item_map)

# Crear la matriz dispersa
user_item_matrix = csr_matrix((df["score"], (df["user_index"], df["item_index"])))


In [14]:
# Configurar y entrenar el modelo ALS
model = AlternatingLeastSquares(
    factors=200,        # Más factores para mejorar la representación de usuarios y productos
    regularization=0.01, # Regularización más baja para capturar más datos
    iterations=100       # Más iteraciones para mejorar la convergencia
)

model.fit(user_item_matrix.T)


c:\Users\Matias\anaconda3\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 0.0019996166229248047 seconds
  warnings.warn(


  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
# Seleccionar un usuario con más compras (el segundo más frecuente en este caso)
user_id = df["user_id"].value_counts().index[1]  
user_idx = user_map.get(user_id)  # Convertir user_id a user_index

# Crear un diccionario inverso para mapear índices internos a product_id
reverse_item_map = {v: k for k, v in item_map.items()}

# Verificar que el usuario existe en el modelo y tiene datos en la matriz
if user_idx is not None and user_idx < model.user_factors.shape[0]:
    if user_item_matrix[user_idx].nnz > 0:  # Verifica que el usuario tiene interacciones
        # Generar recomendaciones
        item_indices, scores = model.recommend(user_idx, user_item_matrix[user_idx], N=5)

        # Convertir índices internos a product_id
        recommended_items = [reverse_item_map[i] for i in item_indices if i in reverse_item_map]

        print(f"Recomendaciones para el usuario {user_id}: {recommended_items}")
    else:
        print(f"El usuario {user_id} no tiene suficientes interacciones en la matriz.")
else:
    print(f"El usuario {user_id} no tiene datos suficientes en el modelo.")


El usuario 177197 no tiene datos suficientes en el modelo.


In [17]:
print(f"Tamaño del modelo (usuarios entrenados): {model.user_factors.shape[0]}")
print(f"Índice del usuario en el modelo: {user_idx}")



Tamaño del modelo (usuarios entrenados): 134
Índice del usuario en el modelo: 11665
